In [17]:
from warnings import filterwarnings as fw; fw('ignore')
import pandas as pd
import pickle

def processing(df):
    ques_id_to_drop = [25131, 25132, 25133, 25134, 25140, 25141, 25142, 25143, 25144]
    
    df = df[df['SectionName'] == 'PMaps Sales Orientation']
    df = df[['CandidateUniqueId', 'OriginalQuestionId', 'OriginalOptionId']]
    df = df[~df['OriginalQuestionId'].isin(ques_id_to_drop)]
    df = df.groupby(['CandidateUniqueId', 'OriginalQuestionId']).first().reset_index()
    df_pivoted = df.pivot(index='CandidateUniqueId', columns='OriginalQuestionId', values='OriginalOptionId')
    df_pivoted = df_pivoted.fillna(0).astype(int)
    encoded_df = pd.get_dummies(df_pivoted, prefix_sep='_', columns=df_pivoted.columns)
    encoded_df = encoded_df[[col for col in encoded_df.columns if not col.endswith('_0')]]
    new_df = pd.DataFrame(index=df_pivoted.index, columns=encoded_df.columns, dtype=int).fillna(0)
    new_df.update(encoded_df)

    return new_df.copy(deep=True)

model = pickle.load(open('model.pkl', 'rb'))

df = pd.read_csv('test.csv')
df = processing(df)

preds = model.predict(df)
probs = model.predict_proba(df)
probs = list(map(lambda x: round(max(x),2), probs))

df_out = df.copy(deep=True)
df_out['Performance'] = list(map(lambda x: x.split('_')[0] , preds))
df_out['Attrition'] = list(map(lambda x: x.split('_')[1] , preds))
df_out['Probability'] = probs

df_out = df_out[['Performance', 'Attrition', 'Probability']]
df_out.to_csv('Predictions.csv')
df_out.head()

,Performance,Attrition,Probability
CandidateUniqueId,,,
1048217,High,Active,0.46
1048218,High,Active,0.82
1048225,Low,Active,0.71
1048241,High,Active,0.54
1048246,Low,Active,0.80
